In [1]:
import fastai
import random
import torch

from fastai.tabular import *

In [2]:
%load_ext autoreload
%autoreload 1
%aimport logs

In [3]:
# Check if we're using the GPU.
torch.cuda.get_device_name(0)

'GeForce GTX 1080'

This next section is training the model. You don't have to do this if the model is already trained.

In [13]:
fname = logs.character_filename("IRONCLAD")
df = pd.read_csv(fname)
df.head()

,Character,Floor,Deck Size,Choice1,Choice2,Choice3,Picked,A Thousand Cuts,A Thousand Cuts+1,AbeCurse,...,Wish+1,Worship,Worship+1,Wraith Form v2,Wraith Form v2+1,WreathOfFlame,WreathOfFlame+1,Writhe,Zap,Zap+1
0,IRONCLAD,1,11,Whirlwind,Twin Strike,Clothesline,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,IRONCLAD,3,11,Seeing Red,Clothesline,Havoc,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,IRONCLAD,4,12,Pummel,Second Wind,Warcry,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,IRONCLAD,5,13,Iron Wave,Blood for Blood,Limit Break,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,IRONCLAD,7,14,Armaments,Anger,Headbutt,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
size = len(df)
procs = [Categorify, Normalize]
dep_var = "Picked"
cat_names = ["Character", "Choice1", "Choice2", "Choice3"]
valid_idx = sorted(random.sample(list(range(size)), int(size / 5)))

In [22]:
data = TabularDataBunch.from_df(fname, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)

In [23]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [24]:
learn.fit_one_cycle(1, 0.01)

epoch     train_loss  valid_loss  accuracy  time    
0         1.055047    743018.2500000.555319  00:24     


In [26]:
learn.fit_one_cycle(1, 0.001)

epoch     train_loss  valid_loss  accuracy  time    
0         1.014597    190482.8750000.559508  00:23     


In [49]:
learn.export(logs.cachefile("ironclad.learn"))

This next section is testing out the model on sample data. The `load_learner` call is loading a cached learner instead of training a new one.

In [4]:
learn = load_learner(logs.CACHE, "ironclad.learn")

In [30]:
deck = ["Strike_R"] * 3 + ["Defend_R"] * 4 + [
    "Bash+1", "Anger", "AscendersBane", "Disarm+1", "Headbutt", "Evolve", "Impervious", "Whirlwind+1", "Shockwave",
    "Demon Form+1", "Heavy Blade+1", "Dark Embrace+1", "Headbutt+1", "Sever Soul+1", "Clothesline+1", "Corruption"]
floor = 36
choices = ["Perfected Strike+1", "Sword Boomerang+1", "Armaments"]
testcsv = logs.mini_csv("IRONCLAD", floor, deck, choices)
testf = pd.read_csv(testcsv)
learn.predict(testf.iloc[0])

(Category tensor(3), tensor(3), tensor([0.0303, 0.4401, 0.0366, 0.4930]))